- week 5

    - remove storage corssborad

    - firm physical interruptible definition

    - how to change kw/h to m3/h



# about dataset
## entsog series:
    * entsog_2019_dataset : selbst extract dataset from entsog capacity talbe 2019
    * lisa's table: same source like talbe above
    * api table: firm capacity firm+interruptable capacity
    
## IGG dataset:
    * IGG+norway dataset+ lisa's convert equation

In [1]:
import pandas as pd
import re
from tqdm import tqdm
import time
from multiprocessing import Pool
import multiprocessing

original_hour = 'https://transparency.entsog.eu/api/v1/operationaldatas.csv?pointDirection={}&from=2019-10-01&to=2020-04-01&indicator=Physical%20Flow&periodType=hour&timezone=CET&limit=-1&dataset=1'

firm_interrupt_hour='https://transparency.entsog.eu/api/v1/operationalData.csv?pointDirection={}&from=2020-01-01&to=2020-10-13&indicator=Firm%20Technical,Interruptible%20Total&periodType=hour&timezone=CET&limit=-1&dataset=1'

interrupt_hour='https://transparency.entsog.eu/api/v1/operationalData.csv?pointDirection={}&from=2020-01-01&to=2020-10-13&indicator=Interruptible%20Total&periodType=hour&timezone=CET&limit=-1&dataset=1'

firm_hour='https://transparency.entsog.eu/api/v1/operationalData.csv?pointDirection={}&from=2020-01-01&to=2020-10-13&indicator=Firm%20Technical&periodType=hour&timezone=CET&limit=-1&dataset=1'
#kWh/h
# or kWh/d
#need divide 10e6

In [2]:
# compare data from lisa
#-----------------------


compare_data = pd.read_excel('Capacities_ENTSOG_2020.xlsx')
compare_data.set_index('Unnamed: 0', inplace=True)
m=compare_data.stack().reset_index()
m.columns=['From','To','V']

# GWh/d

In [3]:
#load data from api
#------------------


operationaldata=pd.read_csv('https://transparency.entsog.eu/api/v1/Interconnections.csv?limit=-1')
fullpointdata=pd.read_csv('https://transparency.entsog.eu/api/v1/connectionpoints.csv?limit=-1',index_col=False)
direction=pd.read_csv('https://transparency.entsog.eu/api/v1/operatorpointdirections.csv?limit=-1')

# only EU
simple_direction_dataset = direction[['id', 'pointKey',
                                                                                 'pointLabel', 'directionKey',
                                                                                 'relatedPoints', 'tsoItemIdentifier',
                                                                                 'tpTsoCountry', 'adjacentCountry',
                                                                                 'tpEURelationship', 'tpCrossBorderPointType',
                                                                                 'tpTsoGCVMax','virtualReverseFlow']].copy()
# clean id ,remove some strange things in string
simple_direction_dataset.id = simple_direction_dataset.id.str[1:].str.lower()

# what id looks like
# de-tso-0009itp-00047exit


# solve id not so clean problem

simple_direction_dataset.id = simple_direction_dataset.id.str.replace(
    'exit', 'split')
simple_direction_dataset.id = simple_direction_dataset.id.str.replace(
    'entry', 'split')
simple_direction_dataset.id = simple_direction_dataset.id.str.split(
    'split').str[0]
simple_direction_dataset.id = simple_direction_dataset.id + \
    simple_direction_dataset.directionKey

# limit to crossborder
p = simple_direction_dataset[simple_direction_dataset.adjacentCountry !=
                             simple_direction_dataset.tpTsoCountry]

#only keep physical direction flow
#virtualReverseFlow dont have nan
p=p[p.virtualReverseFlow=='No']


len(p)

658

In [4]:
direction[direction.tsoItemIdentifier=='21Z0000000004839']

,pointKey,pointLabel,operatorKey,tsoEicCode,operatorLabel,directionKey,validFrom,validTo,hasData,isVirtualizedCommercially,...,tpTsoBalancingZone,tpCrossBorderPointType,tpEURelationship,connectedOperators,adjacentTsoEic,adjacentOperatorKey,adjacentCountry,adjacentZones,id,dataSet
1925,ITP-00535,Deutschneudorf EUGAL Brandov,CZ-TSO-0001,21X000000001304L,NET4GAS,entry,NaN,NaN,1.0,NaN,...,Czech,Cross-Border EU|EU,within EU,GASCADE,21X-DE-H-A0A0A-L,DE-TSO-0001,DE,GASPOOL,5CZ-TSO-0001ITP-00535entryDE-TSO-0001,5
1926,ITP-00535,Deutschneudorf EUGAL Brandov,CZ-TSO-0001,21X000000001304L,NET4GAS,entry,NaN,NaN,1.0,NaN,...,Czech,Cross-Border EU|EU,within EU,ONTRAS Gastransport GmbH,21X-DE-F-A0A0A-2,DE-TSO-0003,DE,GASPOOL,5CZ-TSO-0001ITP-00535entryDE-TSO-0003,5
1927,ITP-00535,Deutschneudorf EUGAL Brandov,CZ-TSO-0001,21X000000001304L,NET4GAS,entry,NaN,NaN,1.0,NaN,...,Czech,Cross-Border EU|EU,within EU,GUD,21X-DE-D-A0A0A-K,DE-TSO-0005,DE,GASPOOL,5CZ-TSO-0001ITP-00535entryDE-TSO-0005,5
1928,ITP-00535,Deutschneudorf EUGAL Brandov,CZ-TSO-0001,21X000000001304L,NET4GAS,entry,NaN,NaN,1.0,NaN,...,Czech,Cross-Border EU|EU,within EU,Fluxys Deutschland,21X0000000012388,DE-TSO-0018,DE,GASPOOL,5CZ-TSO-0001ITP-00535entryDE-TSO-0018,5
1929,ITP-00535,Deutschneudorf EUGAL Brandov,DE-TSO-0001,21X-DE-H-A0A0A-L,GASCADE Gastransport,exit,NaN,NaN,1.0,NaN,...,GASPOOL,Cross-Border EU|EU,within EU,N4G,21X000000001304L,CZ-TSO-0001,CZ,Czech,5DE-TSO-0001ITP-00535exitCZ-TSO-0001,5
1930,ITP-00535,Deutschneudorf EUGAL Brandov,DE-TSO-0003,21X-DE-F-A0A0A-2,ONTRAS,exit,NaN,NaN,1.0,NaN,...,GASPOOL,Cross-Border EU|EU,within EU,N4G,21X000000001304L,CZ-TSO-0001,CZ,Czech,5DE-TSO-0003ITP-00535exitCZ-TSO-0001,5
1931,ITP-00535,Deutschneudorf EUGAL Brandov,DE-TSO-0005,21X-DE-D-A0A0A-K,Gasunie Deutschland,exit,NaN,NaN,1.0,NaN,...,GASPOOL,Cross-Border EU|EU,within EU,N4G,21X000000001304L,CZ-TSO-0001,CZ,Czech,5DE-TSO-0005ITP-00535exitCZ-TSO-0001,5
1932,ITP-00535,Deutschneudorf EUGAL Brandov,DE-TSO-0018,21X0000000012388,Fluxys Deutschland,exit,NaN,NaN,1.0,NaN,...,GASPOOL,Cross-Border EU|EU,within EU,N4G,21X000000001304L,CZ-TSO-0001,CZ,Czech,5DE-TSO-0018ITP-00535exitCZ-TSO-0001,5
1933,ITP-00535,Deutschneudorf EUGAL Brandov,CZ-TSO-0001,21X000000001304L,NET4GAS,exit,NaN,NaN,1.0,NaN,...,Czech,Cross-Border EU|EU,within EU,GUD,21X-DE-D-A0A0A-K,DE-TSO-0005,DE,GASPOOL,5CZ-TSO-0001ITP-00535exitDE-TSO-0005,5
1934,ITP-00535,Deutschneudorf EUGAL Brandov,CZ-TSO-0001,21X000000001304L,NET4GAS,exit,NaN,NaN,1.0,NaN,...,Czech,Cross-Border EU|EU,within EU,Fluxys Deutschland,21X0000000012388,DE-TSO-0018,DE,GASPOOL,5CZ-TSO-0001ITP-00535exitDE-TSO-0018,5


In [5]:
#list of transmission point
#--------------------------

transmission_point_list=list(operationaldata[operationaldata.fromInfrastructureTypeLabel=='Transmission'].pointKey.unique())

# Further filtering about point
## problem only fullpointdata have type of point
    * but direction stilll have some point that not show up
        ['UGS-00273', 'ITP-00111', 'UGS-00050', 'ITP-00494', 'DIS-00003',
       'LNG-00003', 'VTP-00030', 'ITP-00062', 'ITP-00162', 'ITP-00255',
       'ITP-00037', 'ITP-00114', 'ITP-00147', 'ITP-00519', 'ITP-00131',
       'ITP-00125', 'ITP-00026', 'ITP-00034', 'ITP-00075', 'ITP-00105',
       'UGS-00292', 'UGS-00293', 'UGS-00296', 'ITP-00056', 'ITP-00073',
       'ITP-00083', 'ITP-00081', 'ITP-00086', 'ITP-00109', 'ITP-00188',
       'ITP-00491', 'ITP-00057', 'ITP-00068', 'ITP-00006', 'ITP-00031',
       'ITP-00047', 'ITP-00060', 'ITP-00066', 'ITP-00069', 'ITP-00080',
       'ITP-00108', 'ITP-00126', 'UGS-00359', 'UGS-00365', 'ITP-00211',
       'ITP-00250', 'ITP-00251', 'ITP-00452', 'ITP-00247', 'ITP-00297',
       'ITP-00451', 'ITP-00454', 'ITP-00053', 'ITP-00100', 'LNG-00025',
       'LNG-00020', 'ITP-00495', 'ITP-00160', 'ITP-00161', 'ITP-00301',
       'ITP-00302', 'UGS-00253', 'ITP-00431', 'ITP-00432', 'ITP-00433',
       'ITP-00434', 'UGS-00182', 'LNG-00053', 'LNG-00054', 'LNG-00055',
       'UGS-00280', 'UGS-00281', 'UGS-00282', 'UGS-00405', 'UGS-00406',
       'UGS-00407', 'UGS-00408', 'UGS-00409', 'ITP-00496']
        not in fullpointdata list
    
    * according to statistic data ITP is mostly transmission and UGS is mostly storage
## solving    
    * method to filter:
        - choose all points in direction, which are transmission type in fullpointdata
        
        - points that not show up in fullpointdata, according to first three code of pointkey to filter
            * how : keep ITP, LNG

In [6]:
# relationship between point types and first three point key 
fullpointdata.reset_index(inplace=True)
fullpointdata['kurz']=fullpointdata['pointKey'].str[:3]
fullpointdata['number']=1
fullpointdata[['kurz','infrastructureLabel','number']].groupby(['infrastructureLabel','kurz']).sum()

number
infrastructureLabel kurz        
DIS                 DIS      169
                    FNC        1
FNC                 DIS       16
                    FNC       38
ITP                 ITP      247
                    LNG        1
                    UGS        1
LNG                 ITP        1
                    LNG       36
                    VTP        1
PRD                 FNC        1
                    ITP        1
                    PRD       41
UGS                 ITP        2
                    UGS      169
VTP                 VTP       32

In [7]:
# point should in transmission_point_list or start with ITP or LNG
ITP_LNG_in_p=list(p[p.pointKey.str[:3].isin(['ITP','LNG'])].pointKey.unique())
select_list=list(set(transmission_point_list+ITP_LNG_in_p))
p_after_selected=p[p.pointKey.isin(select_list)]
print('before selected: ', len(p),'\n after selected:',len(p_after_selected))


#create geodata
p_with_geodata=p_after_selected.merge(fullpointdata[['pointKey','tpMapX','tpMapY']].drop_duplicates(),left_on='pointKey',right_on='pointKey',how='left')

#there is duplicate, remove it

p_with_geodata.drop_duplicates(inplace=True)

print('after drop duplicate:',len(p_with_geodata))

before selected:  658 
 after selected: 648
after drop duplicate: 584


In [8]:
#id also unique
len(p_with_geodata.id.unique())

580

In [9]:
#load data and clean data
#------------------------


entsog_2019_dataset=pd.read_excel('Capacities for Transmission Capacity Map RTS008_NS - DWH_final.xlsx',sheet_name='Capacity Map',skiprows=range(11))

#clean dataset drop nan
entsog_2019_dataset.dropna(axis=1,how='all',inplace=True)

entsog_2019_dataset.dropna(axis=0,how='all',inplace=True)

#-------choose only cross border capacity
entsog_2019_dataset.reset_index(drop=True,inplace=True)
#entsog_2019_dataset[entsog_2019_dataset.Number.notna()&entsog_2019_dataset.iloc[:,1:].isna().T.all()]
#1-172 crossborder capacity with eu export to on eu
#193-238 corssborder with no eu export to eu
#[193:239]
entsog_2019_dataset=entsog_2019_dataset.iloc[list(range(1,173))+list(range(193,239))]

#only keep rows with capacity
entsog_2019_dataset=entsog_2019_dataset[entsog_2019_dataset['Technical physical capacity (GWh/d)'].notna()&entsog_2019_dataset['Technical physical capacity (GWh/d)']>0]

#fill point name
entsog_2019_dataset.Point.fillna(method='ffill',inplace=True)

#remove viural pipe
entsog_2019_dataset=entsog_2019_dataset[entsog_2019_dataset['Unnamed: 3'].isna()]


entsog_2019_dataset.dropna(axis=1,how='all',inplace=True)

#choose useful data
entsog_2019_dataset=entsog_2019_dataset[['Point','Technical physical capacity (GWh/d)','From Identifier','To Identifier','From CC','To CC']]

#drop that a country that not in eu and also some capacity to storage
entsog_2019_dataset=entsog_2019_dataset[entsog_2019_dataset['From CC'].notna()&entsog_2019_dataset['To CC'].notna()]

#capacity must not be nan
entsog_2019_dataset=entsog_2019_dataset[entsog_2019_dataset['Technical physical capacity (GWh/d)'].notna()]

#clean point name
entsog_2019_dataset.Point=entsog_2019_dataset.Point.str.split('\\n').str[0]
# .str means not string but means dealing with single cell not as series


In [10]:
#add point key information
#-------------------------


entsog_2019_dataset=entsog_2019_dataset.merge(operationaldata[['pointKey','pointLabel','pointTpMapX','pointTpMapY']].drop_duplicates(),left_on='Point',right_on='pointLabel',how='left')

entsog_2019_dataset.fillna('NULL',inplace=True)
#replace nan with 'NULL'
#nan is hard to deal with for single cell value


#deal with 38ZEEG
def deal_with_38ZEEG(df):
    if df['From Identifier']=='38ZEEG-0007107-I' or df['To Identifier']=='38ZEEG-0007107-I':
        df['pointKey']=fullpointdata.set_index('pointEicCode').loc['38ZEEG-0007107-I']['pointKey']
        df['Point']=fullpointdata.set_index('pointEicCode').loc['38ZEEG-0007107-I']['pointKey']
        df['pointLabel']=fullpointdata.set_index('pointEicCode').loc['38ZEEG-0007107-I']['pointLabel']
        df['pointTpMapX']=fullpointdata.set_index('pointEicCode').loc['38ZEEG-0007107-I']['tpMapX']
        df['pointTpMapY']=fullpointdata.set_index('pointEicCode').loc['38ZEEG-0007107-I']['tpMapY']
    return df
entsog_2019_dataset=entsog_2019_dataset.apply(deal_with_38ZEEG,axis=1)

try:
    entsog_2019_dataset.drop('pointLabel',axis=1,inplace=True)
except KeyError:
    pass
entsog_2019_dataset.columns=['Point','Capacity(GWh/d)','From_ID','To_ID','From','To','PointKey','Longitude','Latitude']


#show the rows not match
entsog_2019_dataset[entsog_2019_dataset.PointKey=='NULL']

,Point,Capacity(GWh/d),From_ID,To_ID,From,To,PointKey,Longitude,Latitude
82,Láb (SK) / Láb IV (AT),138.3,NULL,NULL,AT,SK,NULL,NULL,NULL
83,Láb (SK) / Láb IV (AT),138.3,NULL,NULL,SK,AT,NULL,NULL,NULL


In [11]:
#statistic
#---------

#remove inside country point
entsog_2019_dataset=entsog_2019_dataset[entsog_2019_dataset.From!=entsog_2019_dataset.To]

#remove not transmission type points, store in another dataframe
only_transmission_entsog_2019=entsog_2019_dataset[entsog_2019_dataset.PointKey.isin(transmission_point_list)]

self_extract_data=only_transmission_entsog_2019.groupby(['From','To']).sum()['Capacity(GWh/d)']

self_extract_data=self_extract_data.reset_index()
self_extract_data

,From,To,Capacity(GWh/d)
0,AT,DE,635.4456
1,AT,HU,153.1000
2,AT,IT,1148.8000
3,AT,SI,112.5000
4,AT,SK,246.5000
...,...,...,...
74,UA,PL,135.6000
75,UA,RO,743.2000
76,UA,SK,2277.6000
77,UK,BE,651.7000


In [12]:
#compare table from same source
compare_table=m.merge(self_extract_data,left_on=['From','To'],right_on=['From','To'],how='left')
compare_table['diff_percent']=(compare_table.V-compare_table['Capacity(GWh/d)'])/compare_table.V
compare_table.diff_percent=compare_table.diff_percent
compare_table[(compare_table.diff_percent.abs()>0.1)|compare_table.diff_percent.isna()]

#table from lisa have more information
#selbst extract data from same source 62.5% are almost the same (different<10%)
#selbst extract data are often have larger capacity then lisa's table, when difference are greater than 10%

,From,To,V,Capacity(GWh/d),diff_percent
0,AT,DE,390.0,635.445600,-0.629348
6,BE,FR,850.0,230.000000,0.729412
7,BE,LU,49.0,NaN,NaN
17,CZ,DE,1242.0,921.824300,0.257790
19,CZ,SK,914.0,1009.280000,-0.104245
20,DE,AT,360.0,611.984000,-0.699956
24,DE,DK,124.0,166.500000,-0.342742
27,DE,NL,503.0,664.510115,-0.321094
28,DE,PL,234.0,184.800000,0.210256
30,DK,SE,81.0,NaN,NaN


In [13]:
len(entsog_2019_dataset[['PointKey','From','To']].drop_duplicates())-len(entsog_2019_dataset)
#this three parameters value combination is unique key

0

## Physical hour data

In [14]:
'''
Multiprocessor version
'''
def get_data_hour(inputs):
    df=inputs[0]
    api_url=inputs[1]
    process_number=inputs[2]
    answer=[]
    notsuccessful=0
    for i in df.id:
        #print('-----')
        trytime=0
        while trytime<2:
            try:
                url=api_url.format(i)
                #get data
                temp_table=pd.read_csv(url)
                #united id 
                temp_table.id=i
                #get max Physically flow
                temp_table=temp_table[['id','value']].groupby('id').max()
                #set index
                temp_table.reset_index(inplace=True)

                answer.append(temp_table)
                break
            except:
                trytime+=1
                if trytime==2:
                    #record how many rows fail to get data from server
                    notsuccessful+=1

    print('process{} is finish with '.format(process_number),notsuccessful,'not successful')
    #return a dataset
    return pd.concat(answer)


def split_df(df,Processors=2):
    
    df = df.sample(frac=1).reset_index(drop=True)
    
    stride=(len(df)//Processors)+1
    
    return [df.iloc[x*stride:x*stride+stride,:].copy() for x in range(Processors)]




def fix_direction(dataframe):
    if dataframe['directionKey']=='entry':
        temp_value=dataframe['tpTsoCountry']
        dataframe['tpTsoCountry']=dataframe['adjacentCountry']
        dataframe['adjacentCountry']=temp_value
    return dataframe
#direction always is from tp to adj
#which means if is exit then dont need to change, entry need to be change
#enrty means gas enter tpTso


#split dataset which can be use three times
#p_split=split_df(p,multiprocessing.cpu_count())

# only firm capacity daily 

In [15]:
#load from internet
#------------------
processors=multiprocessing.cpu_count()
p_split=split_df(p_with_geodata,processors)
#get data
args=[[i,original_hour,x] for x,i in enumerate(p_split)]

with Pool(processes=processors) as pool:
    physical_capacity=pool.map(get_data_hour,args)
    
physical_capacity=pd.concat(physical_capacity)            

#physical_capacity=get_data_hour(p_with_geodata,firm_hour)
physical_capacity.to_csv('physical_capacity',index=False)

#load from local
#-----------------
#physical_capacity=pd.read_csv('physical_capacity')
#physical_capacity=physical_capacity[physical_capacity.id.isin(real_flow_list)]
#physical_capacity.drop('Unnamed: 0',axis=1,inplace=True)


physical_capacity=p_with_geodata.merge(physical_capacity,left_on='id',right_on='id')

physical_capacity=physical_capacity.apply(fix_direction,axis=1)

physical_capacity.drop('directionKey',axis=1,inplace=True)


physical_capacity_country_level=physical_capacity[['tpTsoCountry','adjacentCountry','value']].groupby(by=['tpTsoCountry','adjacentCountry']).sum().reset_index()

#from kWh/h to GWh/d
physical_capacity_country_level.value=physical_capacity_country_level.value/10e6

physical_capacity_country_level.columns=['tpTsoCountry','adjacentCountry','firm_value']

physical_capacity_country_level.head()

process7 is finish with  38 not successful
process6 is finish with  37 not successful
process0 is finish with  40 not successful
process3 is finish with  38 not successful
process1 is finish with  38 not successful
process2 is finish with  46 not successful
process5 is finish with  26 not successful
process4 is finish with  30 not successful


,tpTsoCountry,adjacentCountry,firm_value
0,AT,DE,5.516769
1,AT,HU,0.657110
2,AT,IT,10.347393
3,AT,SI,0.775748
4,AT,SK,0.000000


# something datat physical data (because of pipe in pipe)
# firm data some times have missing data
# physical data will be processed by entsog, one point same direction(from country a to country b) all operators willl have almost the same physical flow
# so physical  data need to be process twice


# firm capacity 

# some times interruptable data have infintie

# final data will be groupby pointkey+fromcountry+tocountry

In [ ]:
#some times firm capacity have no data,which means 0, so it is better to use firm with physical capacity

In [16]:
#internet
#--------------
processors=multiprocessing.cpu_count()
p_split=split_df(p_with_geodata,processors)
#get data
args=[[i,firm_hour,x] for x,i in enumerate(p_split)]

with Pool(processes=processors) as pool:
    firm_capacity=pool.map(get_data_hour,args)
    
firm_capacity=pd.concat(firm_capacity)            

#firm_capacity=get_data_hour(p_with_geodata,firm_hour)
firm_capacity.to_csv('firm_capacity',index=False)

#local
#---------------
#firm_capacity=pd.read_csv('firm_capacity')
#firm_capacity=firm_capacity[firm_capacity.id.isin(real_flow_list)]
#firm_capacity.drop('Unnamed: 0',axis=1,inplace=True)


firm_capacity=p_with_geodata.merge(firm_capacity,left_on='id',right_on='id')

firm_capacity=firm_capacity.apply(fix_direction,axis=1)

firm_capacity.drop('directionKey',axis=1,inplace=True)


firm_capacity_country_level=firm_capacity[['tpTsoCountry','adjacentCountry','value']].groupby(by=['tpTsoCountry','adjacentCountry']).sum().reset_index()

#from kWh/h to GWh/d
firm_capacity_country_level.value=firm_capacity_country_level.value/10e6

firm_capacity_country_level.columns=['tpTsoCountry','adjacentCountry','firm_value']

firm_capacity_country_level.head()

process0 is finish with  39 not successful
process4 is finish with  47 not successful
process3 is finish with  38 not successful
process1 is finish with  47 not successful
process5 is finish with  47 not successful
process2 is finish with  34 not successful
process6 is finish with  48 not successful
process7 is finish with  38 not successful


,tpTsoCountry,adjacentCountry,firm_value
0,AT,DE,5.645642
1,AT,HU,0.637830
2,AT,IT,5.001497
3,AT,SI,1.049082
4,AT,SK,1.027200


interruptable capacity
------------

they are different reccords with different labels, just easy to sum up

In [17]:
processors=multiprocessing.cpu_count()
p_split=split_df(p_with_geodata,processors)
#get data
args=[[i,interrupt_hour,x] for x,i in enumerate(p_split)]

with Pool(processes=processors) as pool:
    inter_capacity=pool.map(get_data_hour,args)
    
inter_capacity=pd.concat(inter_capacity)            

inter_capacity.to_csv('inter_capacity',index=False)





#inter_capacity=pd.read_csv('inter_capacity')
#inter_capacity=inter_capacity[inter_capacity.id.isin(real_flow_list)]
#inter_capacity.drop('Unnamed: 0',axis=1,inplace=True)



inter_capacity=p_with_geodata.merge(inter_capacity,left_on='id',right_on='id')


inter_capacity=inter_capacity.apply(fix_direction,axis=1)

inter_capacity.drop('directionKey',axis=1,inplace=True)

inter_capacity_country_level=inter_capacity[['tpTsoCountry','adjacentCountry','value']].groupby(by=['tpTsoCountry','adjacentCountry']).sum().reset_index()

#from kWh/h to GWh/d
inter_capacity_country_level.value=inter_capacity_country_level.value/10e6

inter_capacity_country_level.value=inter_capacity_country_level.value.round()

inter_capacity_country_level.columns=['tpTsoCountry','adjacentCountry','inter_value']

inter_capacity_country_level.head()

process5 is finish with  44 not successful
process3 is finish with  41 not successful
process1 is finish with  38 not successful
process6 is finish with  43 not successful
process2 is finish with  40 not successful
process7 is finish with  43 not successful
process0 is finish with  41 not successful
process4 is finish with  45 not successful


,tpTsoCountry,adjacentCountry,inter_value
0,AT,DE,2.0
1,AT,HU,1.0
2,AT,IT,4.0
3,AT,SI,1.0
4,AT,SK,1.0


In [38]:
#there is problem, same different direction id means the same flow and the same pipes, need to drop duplicate
#solve remove id and drop duplicates
def drop_id_drop_duplicate(df):
    df.drop('id',axis=1,inplace=True)
    len_before=len(df)
    df.drop_duplicates(inplace=True)
    len_after=len(df)
    print('before: ',len_before,' after: ',len_after)
    return df

physical_capacity=drop_id_drop_duplicate(physical_capacity)
firm_capacity=drop_id_drop_duplicate(firm_capacity)
inter_capacity=drop_id_drop_duplicate(inter_capacity)

before:  233  after:  230
before:  206  after:  205
before:  209  after:  205


In [44]:
x=physical_capacity[['pointKey','tpTsoCountry','adjacentCountry']].copy()
x['m']=1
x=x.groupby(['pointKey','tpTsoCountry','adjacentCountry']).count().reset_index()
x.sort_values('m')

,pointKey,tpTsoCountry,adjacentCountry,m
0,ITP-00006,AT,DE,1
102,ITP-00162,AT,SK,1
103,ITP-00162,SK,AT,1
104,ITP-00168,AT,SK,1
105,ITP-00168,SK,AT,1
...,...,...,...,...
55,ITP-00076,DE,NL,2
114,ITP-00244,NL,DE,2
84,ITP-00123,CZ,DE,2
10,ITP-00019,AT,DE,3


In [20]:
final_capacity=inter_capacity.merge(firm_capacity,how='outer',left_on=['tpTsoCountry','adjacentCountry'],right_on=['tpTsoCountry','adjacentCountry'])
#final_capacity=final_capacity.merge(physical_capacity,how='outer',left_on=['tpTsoCountry','adjacentCountry'],right_on=['tpTsoCountry','adjacentCountry'])

In [22]:
final_capacity['day']=final_capacity.firm_value*24

In [23]:
def add_compare(dataframe):
    return compare_data[dataframe['adjacentCountry']][dataframe['tpTsoCountry']]
final_capacity['compare_value']=final_capacity.apply(add_compare,axis=1)

In [27]:
ak=final_capacity.merge(self_extract_data,left_on=[ 'tpTsoCountry','adjacentCountry'],right_on=['From','To'],how='outer')
#GWh/d
ak[ak.tpTsoCountry.notna()]

,tpTsoCountry,adjacentCountry,inter_value,firm_value,day,compare_value,From,To,Capacity(GWh/d)
0,AT,DE,3.0,5.645642,135.495418,390.0,AT,DE,635.445600
1,AT,HU,1.0,0.637830,15.307920,153.0,AT,HU,153.100000
2,AT,IT,4.0,5.001497,120.035926,1149.0,AT,IT,1148.800000
3,AT,SI,1.0,1.049082,25.177973,113.0,AT,SI,112.500000
4,AT,SK,1.0,1.027200,24.652800,247.0,AT,SK,246.500000
5,BE,DE,2.0,4.734762,113.634278,290.0,BE,DE,289.500000
6,BE,FR,0.0,6.791195,162.988680,850.0,BE,FR,230.000000
7,BE,LU,0.0,0.000000,0.000000,49.0,NaN,NaN,NaN
8,BE,NL,201.0,104.591174,2510.188183,394.0,BE,NL,393.000000
9,BE,UK,0.0,4.035591,96.854186,803.0,BE,UK,803.400000


In [ ]:
entsog family

igg

change the unit

pdf about diff dataset

capacity per hour + start and end highest resolution (city longtitude and lagtitude)

try with the different pressure samll area ;country; all 

check 

segement error

In [ ]:
#physical
#arg=[original_hour for i in p_split]

#with Pool(multiprocessing.cpu_count()) as pool:
#    physical_capacity=pool.starmap(get_data_hour,zip(p_split,arg))


#-----------------------
##get data from internet
#physical_capacity=get_data_hour(p,original_hour)
#physical_capacity.to_csv('physical_capacity')

#-----------------------
#load from local
physical_capacity=pd.read_csv('physical_capacity')
physical_capacity=physical_capacity[physical_capacity.id.isin(real_flow_list)]
physical_capacity.drop('Unnamed: 0',axis=1,inplace=True)
physical_capacity.drop_duplicates(inplace=True)

physical_capacity_with_geodata=p_with_geodata.drop_duplicates().merge(physical_capacity,left_on='id',right_on='id')
physical_capacity_with_geodata=physical_capacity_with_geodata.apply(fix_direction,axis=1)


physical_capacity=p.merge(physical_capacity,left_on='id',right_on='id')[['directionKey','tpTsoCountry','adjacentCountry','value']]



physical_capacity=physical_capacity.apply(fix_direction,axis=1)

physical_capacity.drop('directionKey',axis=1,inplace=True)


physical_capacity=physical_capacity.groupby(by=['tpTsoCountry','adjacentCountry']).sum().reset_index()

#from kWh/h to GWh/d
physical_capacity.value=physical_capacity.value/10e6


physical_capacity.columns=['tpTsoCountry','adjacentCountry','physical_value']

physical_capacity.head()